## Pruebas que habría que hacer

- Para cada algoritmo: 
    - Búsqueda de hiperparámetros con cada par de métricas de fitness
    - Búsqueda de hiperparámetros con diferentes proporciones de población/iteraciones (50k llamadas a la función de fitness es el máximo)

Una vez realizada la búsqueda, quedarse con os mejores frentes de cada par de métricas y algoritmo (8 en total) y se hace el estudio comparativo con el ground truth (parte C)


Para comparar Paretos se puede utilizar la medida de volumen del area que forma.

In [4]:
from nsga import NSGA
from moga import MOGA
import networkx as nx
import itertools
from concurrent.futures import ProcessPoolExecutor, as_completed

In [9]:
N = [100]
init = [0, 0.5, 1]
pcross= [0.5, 0.7]
pmut = [0.3, 0.5]
n_iter= [500]
fitness_metrics= [0]
n_tour = [2,4,8]
crossover_op=[0,1,2]

In [ ]:
# Carga del grafo
graph = nx.read_graphml("data/amazon_graph.graphml")

# Renombramos los nodos al rango [0, N]
mapping = {node: i for i, node in enumerate(graph.nodes())}
graph = nx.relabel_nodes(graph, mapping)


In [ ]:
def run_experiment(combination):
    value_sum = 0
    weight_exceeded = 0
    max_value = 0
    for _ in range(num_experiments):
        estrategia = MOGA(graph, *combination)
        pop, old_fitness, pareto = estrategia.evolve()

        #! HAY QUE DISEÑAR UNA FUNCIÓN QUE CALCULE EL AREA DEL PARETO Y DEVOLVER LA MEDIA DE LOS EXPERIMENTOS
        

# Define el número de veces que cada configuración será ejecutada
num_experiments = 10

# Crear todas las combinaciones posibles de hiperparámetros
all_combinations = list(itertools.product(
    N, init, pcross, pmut, n_iter, fitness_metrics, n_tour, crossover_op
))

# Almacenar los resultados
results = []

# Paralelizar la ejecución
with ProcessPoolExecutor() as executor:
    # Iniciar las ejecuciones en paralelo
    future_to_combination = {executor.submit(run_experiment, combination): combination for combination in all_combinations}

    total_combinations = len(all_combinations)
    completed_combinations = 0
    #! REVISAR ESTO CUANDO ESTÉ CREADA LA FUNCIÓN DE AREA
    for future in as_completed(future_to_combination):
        combination, average_value, max_value, weight_exceeded = future.result()
        results.append((combination, average_value, max_value, weight_exceeded))

        completed_combinations += 1
        print(f"Combinación completada: {completed_combinations}/{total_combinations} - {combination} - {average_value} - {max_value} - {weight_exceeded}")
